In [11]:
import pandas as pd
DatosV = pd.read_excel("C:/Users/Felipe Bañol/Documents/GitHub/Metro2.0/DatosV.xlsx")
DatosV["EDAD"] = pd.to_numeric(DatosV["EDAD"], errors="coerce")
# Crear columna indicador
DatosV["Treat"] = DatosV["ESTACION"].isin(["ACEVEDO", "SAN ANTONIO", "SAN JAVIER"]).astype(int)
def marcar_time2022(df):
    periodos = [
        "2022-1", "2022-2", "2022-3", "2022-4",
        "2023-1", "2023-2", "2023-3", "2023-4",
        "2024-1", "2024-2", "2024-3", "2024-4"
    ]
    df["Time2022"] = df["AñoT"].isin(periodos).astype(int)
    return df

In [5]:



# Filtrar por género
DatosM = DatosV[DatosV["GENERO"] == "M"]
DatosF = DatosV[DatosV["GENERO"] == "F"]
print("Masculino:", len(DatosM))
print("Femenino:", len(DatosF))

Masculino: 577
Femenino: 459


In [6]:
DatosM = DatosM[DatosM["AñoT"] >= "2012-1"]
DatosF = DatosF[DatosF["AñoT"] >= "2012-1"]

# Verificar cantidad de registros
print("Masculino recientes:", len(DatosM))
print("Femenino recientes:", len(DatosF))

Masculino recientes: 395
Femenino recientes: 255


In [7]:

import itertools


# 1️⃣ Crear lista de trimestres desde 2012-1 hasta 2023-4 para masculino
años = range(2012, 2024)
trimestres = [1, 2, 3, 4]
todos_trimestres = [f"{año}-{t}" for año, t in itertools.product(años, trimestres)]

# 2️⃣ Obtener lista de estaciones únicas (por ejemplo de DatosM)
estaciones = DatosM["ESTACION"].unique()

# 3️⃣ Crear DataFrame con todas las combinaciones de Estación x AñoT
combinaciones = pd.MultiIndex.from_product([estaciones, todos_trimestres], names=["ESTACION", "AñoT"]).to_frame(index=False)

# 4️⃣ Hacer conteo de ocurrencias en DatosM
conteo = DatosM.groupby(["ESTACION", "AñoT"]).size().reset_index(name="conteo")

# 5️⃣ Hacer merge con todas las combinaciones, rellenando NaN con 0
DatosM_conteo = combinaciones.merge(conteo, on=["ESTACION", "AñoT"], how="left").fillna(0)

# 6️⃣ Asegurarse que 'conteo' sea entero
DatosM_conteo["conteo"] = DatosM_conteo["conteo"].astype(int)

# Ver los primeros registros
print(DatosM_conteo.head())


  ESTACION    AñoT  conteo
0  ACEVEDO  2012-1       0
1  ACEVEDO  2012-2       1
2  ACEVEDO  2012-3       1
3  ACEVEDO  2012-4       0
4  ACEVEDO  2013-1       0


In [8]:
# 1️⃣ Crear lista de trimestres desde 2012-1 hasta 2023-4 para femenino
años = range(2012, 2024)
trimestres = [1, 2, 3, 4]
todos_trimestres = [f"{año}-{t}" for año, t in itertools.product(años, trimestres)]

# 2️⃣ Obtener lista de estaciones únicas (por ejemplo de DatosM)
estaciones_f = DatosF["ESTACION"].unique()

# 3️⃣ Crear DataFrame con todas las combinaciones de Estación x AñoT
combinaciones_f = pd.MultiIndex.from_product([estaciones, todos_trimestres], names=["ESTACION", "AñoT"]).to_frame(index=False)

# 4️⃣ Hacer conteo de ocurrencias en DatosM
conteo_f = DatosF.groupby(["ESTACION", "AñoT"]).size().reset_index(name="conteo")

# 5️⃣ Hacer merge con todas las combinaciones, rellenando NaN con 0
DatosF_conteo = combinaciones_f.merge(conteo, on=["ESTACION", "AñoT"], how="left").fillna(0)
 
# 6️⃣ Asegurarse que 'conteo' sea entero
DatosF_conteo["conteo"] = DatosF_conteo["conteo"].astype(int)

# Ver los primeros registros
print(DatosF_conteo.head())

  ESTACION    AñoT  conteo
0  ACEVEDO  2012-1       0
1  ACEVEDO  2012-2       1
2  ACEVEDO  2012-3       1
3  ACEVEDO  2012-4       0
4  ACEVEDO  2013-1       0


In [9]:
Datos_total = pd.concat([DatosM, DatosF], ignore_index=True)

# 2️⃣ Lista de trimestres desde 2012-1 hasta 2023-4
años = range(2012, 2024)
trimestres = [1, 2, 3, 4]
todos_trimestres = [f"{año}-{t}" for año, t in itertools.product(años, trimestres)]

# 3️⃣ Lista de estaciones únicas en la base combinada
estaciones_total = Datos_total["ESTACION"].unique()

# 4️⃣ Crear DataFrame con todas las combinaciones de Estación x AñoT
combinaciones_total = pd.MultiIndex.from_product([estaciones_total, todos_trimestres], names=["ESTACION", "AñoT"]).to_frame(index=False)

# 5️⃣ Hacer conteo de ocurrencias en la base combinada
conteo_total = Datos_total.groupby(["ESTACION", "AñoT"]).size().reset_index(name="conteo")

# 6️⃣ Merge con todas las combinaciones, rellenando NaN con 0
Datos_total_conteo = combinaciones_total.merge(conteo_total, on=["ESTACION", "AñoT"], how="left").fillna(0)

# 7️⃣ Asegurarse que 'conteo' sea entero
Datos_total_conteo["conteo"] = Datos_total_conteo["conteo"].astype(int)

# Ver los primeros registros
print(Datos_total_conteo.head())

  ESTACION    AñoT  conteo
0  ACEVEDO  2012-1       0
1  ACEVEDO  2012-2       2
2  ACEVEDO  2012-3       1
3  ACEVEDO  2012-4       0
4  ACEVEDO  2013-1       0


In [10]:

DatosV = DatosV[DatosV["AñoT"] >= "2012-1"]

# Lista de trimestres
años = range(2012, 2024)
trimestres = [1, 2, 3, 4]
todos_trimestres = [f"{año}-{t}" for año, t in itertools.product(años, trimestres)]

def generar_conteo(df, nombre):
    """Genera DataFrame de conteo por Estación y AñoT incluyendo trimestres vacíos"""
    estaciones = df["ESTACION"].unique()
    combinaciones = pd.MultiIndex.from_product([estaciones, todos_trimestres], names=["ESTACION", "AñoT"]).to_frame(index=False)
    conteo = df.groupby(["ESTACION", "AñoT"]).size().reset_index(name="conteo")
    resultado = combinaciones.merge(conteo, on=["ESTACION", "AñoT"], how="left").fillna(0)
    resultado["conteo"] = resultado["conteo"].astype(int)
    return resultado

# -----------------------------
# NO JOVENES (EDAD > 28)
# -----------------------------
Datos_no_joven = DatosV[DatosV["EDAD"] > 28]

DatosM_no_joven = generar_conteo(Datos_no_joven[Datos_no_joven["GENERO"]=="M"], "M_no_joven")
DatosF_no_joven = generar_conteo(Datos_no_joven[Datos_no_joven["GENERO"]=="F"], "F_no_joven")
Datos_total_no_joven = generar_conteo(Datos_no_joven, "Total_no_joven")

# -----------------------------
# JOVENES (EDAD <= 28)
# -----------------------------
Datos_joven = DatosV[DatosV["EDAD"] <= 28]

DatosM_joven = generar_conteo(Datos_joven[Datos_joven["GENERO"]=="M"], "M_joven")
DatosF_joven = generar_conteo(Datos_joven[Datos_joven["GENERO"]=="F"], "F_joven")
Datos_total_joven = generar_conteo(Datos_joven, "Total_joven")

# -----------------------------
# Ejemplo: ver primeras filas
print("DatosM_no_joven")
print(DatosM_no_joven.head())

print("Datos_total_joven")
print(Datos_total_joven.head())


DatosM_no_joven
  ESTACION    AñoT  conteo
0  ACEVEDO  2012-1       0
1  ACEVEDO  2012-2       0
2  ACEVEDO  2012-3       0
3  ACEVEDO  2012-4       0
4  ACEVEDO  2013-1       0
Datos_total_joven
        ESTACION    AñoT  conteo
0  12 DE octubre  2012-1       0
1  12 DE octubre  2012-2       0
2  12 DE octubre  2012-3       0
3  12 DE octubre  2012-4       0
4  12 DE octubre  2013-1       0


In [12]:
DatosM_conteo = marcar_time2022(DatosM_conteo)
DatosF_conteo = marcar_time2022(DatosF_conteo)
Datos_total_conteo = marcar_time2022(Datos_total_conteo)

DatosM_no_joven = marcar_time2022(DatosM_no_joven)
DatosF_no_joven = marcar_time2022(DatosF_no_joven)
Datos_total_no_joven = marcar_time2022(Datos_total_no_joven)

DatosM_joven = marcar_time2022(DatosM_joven)
DatosF_joven = marcar_time2022(DatosF_joven)
Datos_total_joven = marcar_time2022(Datos_total_joven)


In [16]:
def agregar_variables(df):
    # 1) Treat: estaciones tratadas
    estaciones_tratadas = ["ACEVEDO", "SAN ANTONIO", "SAN JAVIER"]
    df["Treat"] = df["ESTACION"].isin(estaciones_tratadas).astype(int)

    # 2) Time2020: periodos desde 2020-2 hasta 2024-4
    periodos_2020 = [
        "2020-2", "2020-3", "2020-4",
        "2021-1", "2021-2", "2021-3", "2021-4",
        "2022-1", "2022-2", "2022-3", "2022-4",
        "2023-1", "2023-2", "2023-3", "2023-4",
        "2024-1", "2024-2", "2024-3", "2024-4"
    ]
    df["Time2020"] = df["AñoT"].isin(periodos_2020).astype(int)

    # 3) Time2022: periodos desde 2022-1 hasta 2024-4
    periodos_2022 = [
        "2022-1", "2022-2", "2022-3", "2022-4",
        "2023-1", "2023-2", "2023-3", "2023-4",
        "2024-1", "2024-2", "2024-3", "2024-4"
    ]
    df["Time2022"] = df["AñoT"].isin(periodos_2022).astype(int)

    # 4) TimeT2020 = Treat * Time2020
    df["TimeT2020"] = df["Treat"] * df["Time2020"]

    # 5) TimeT2022 = Treat * Time2022
    df["TimeT2022"] = df["Treat"] * df["Time2022"]

    return df


In [17]:
# Bases originales de conteo
DatosM_conteo = agregar_variables(DatosM_conteo)
DatosF_conteo = agregar_variables(DatosF_conteo)
Datos_total_conteo = agregar_variables(Datos_total_conteo)

# Bases no jóvenes
DatosM_no_joven = agregar_variables(DatosM_no_joven)
DatosF_no_joven = agregar_variables(DatosF_no_joven)
Datos_total_no_joven = agregar_variables(Datos_total_no_joven)

# Bases jóvenes
DatosM_joven = agregar_variables(DatosM_joven)
DatosF_joven = agregar_variables(DatosF_joven)
Datos_total_joven = agregar_variables(Datos_total_joven)
